# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Grabbing and reading the csv made from previous part of assignment and creating a dataframe.

weather_data = "Output_Data/new_cities.csv"

weather_df = pd.read_csv(weather_data)

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tasiilaq,65.6145,-37.6368,17.60,45,54,4.61,GL,1610233700
1,Ushuaia,-54.8000,-68.3000,53.60,54,75,6.91,AR,1610233762
2,Albany,42.6001,-73.9662,28.99,56,100,7.23,US,1610233601
3,Sydney,-33.8679,151.2073,75.99,60,20,6.91,AU,1610234143
4,Zelenoborskiy,66.8450,32.3622,-4.09,93,88,2.59,RU,1610234175


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Using generated g_key from Google API.

gmaps.configure(api_key=g_key)

# Grabbing needed data to construct a heatmap based on Humidity for cities.

locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df['Humidity'].astype(float)

# Creating google map with a heatmap layer based on Humidity.  Setting parameters to show as much of the map as possible.

fig = gmaps.figure(center=(10, 0.1), zoom_level=1.8)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df['Humidity']),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Setting parameters for ideal vacation weather, humidity, etc. and grabbing all cities that match all criteria,
# dropping cities that have null data if any of the paramters were not met.

ideal_weather = weather_df.loc[(weather_df['Max Temp'] > 60) & (weather_df['Max Temp'] < 70) & (weather_df['Wind Speed'] < 5) & (weather_df['Cloudiness'] < 10)]
hotel_df = ideal_weather.dropna()

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,Fortuna,40.5982,-124.1573,60.01,58,1,3.65,US,1610233949
234,Digapahandi,19.3667,84.5833,66.99,61,8,2.64,IN,1610234237
249,Mar Del Plata,-38.0023,-57.5575,69.01,77,0,3.44,AR,1610233751
265,Lalmohan,22.3384,90.7419,69.62,60,2,4.68,BD,1610234244
446,Saldanha,-33.0117,17.9442,66.20,77,0,4.61,ZA,1610233926
469,Harqalah,36.0303,10.5090,66.00,62,0,3.44,TN,1610234302
512,Bilma,18.6853,12.9164,66.15,26,0,2.08,NE,1610233949


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Creating a list containing empty strings equal to the number of cities that meet vacation criteria and
# creating a new emtpy column in the dataframe

hotels = [''] * len(hotel_df.index)
hotel_df.insert(9, 'Hotel Name', hotels, True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
44,Fortuna,40.5982,-124.1573,60.01,58,1,3.65,US,1610233949,
234,Digapahandi,19.3667,84.5833,66.99,61,8,2.64,IN,1610234237,
249,Mar Del Plata,-38.0023,-57.5575,69.01,77,0,3.44,AR,1610233751,
265,Lalmohan,22.3384,90.7419,69.62,60,2,4.68,BD,1610234244,
446,Saldanha,-33.0117,17.9442,66.20,77,0,4.61,ZA,1610233926,
469,Harqalah,36.0303,10.5090,66.00,62,0,3.44,TN,1610234302,
512,Bilma,18.6853,12.9164,66.15,26,0,2.08,NE,1610233949,


In [6]:
# Setting the base url for the google API and setting parameters in a dictionary to find hotels within 5000 meters
# of each city that match the vacation criteria.

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    'radius': 5000,
    "type": "lodging",
    "key": g_key,
}

# Created an empty list to hold coordinates for hotels found.

hotel_coord = []

# Used iterrows to iterate through pandas dataframe for each index/row.

for index, row in hotel_df.iterrows():

    # Get latitude, longitude, and city name from dataframe
    
    lat = row['Lat']
    lon = row['Lng']
    city = row['City']

    # Add location with lat, lon coordinates to params dictionary
    
    params['location'] = f'{lat},{lon}'

    # Assemble url and make API request pulling the hotel names and their coordinates and adding hotel name to dataframe.
    
    try:
        response = requests.get(base_url, params=params).json()
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        hotel_coord.append((response['results'][0]['geometry']['location']['lat'], response['results'][0]['geometry']['location']['lng']))
   
    # If no hotel is found within 5000 meters of city, print that info and add phrase to dataframe
    
    except:
        print(f'No hotel within 5000 meters of {city}')
        hotel_df.loc[index, 'Hotel Name'] = "None within 5000 meters"


In [7]:
# Showing dataframe to confirm filled Hotel Name column.

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
44,Fortuna,40.5982,-124.1573,60.01,58,1,3.65,US,1610233949,Super 8 by Wyndham Fortuna
234,Digapahandi,19.3667,84.5833,66.99,61,8,2.64,IN,1610234237,Tiffin Hotel
249,Mar Del Plata,-38.0023,-57.5575,69.01,77,0,3.44,AR,1610233751,Gran Hotel Mar del Plata
265,Lalmohan,22.3384,90.7419,69.62,60,2,4.68,BD,1610234244,Sundorbon courier service
446,Saldanha,-33.0117,17.9442,66.20,77,0,4.61,ZA,1610233926,Blue Bay Lodge
469,Harqalah,36.0303,10.5090,66.00,62,0,3.44,TN,1610234302,Dar Khadija
512,Bilma,18.6853,12.9164,66.15,26,0,2.08,NE,1610233949,Expeditions Ténére voyages


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Assign the marker layer to a variable.
# Info box content code found https://jupyter-gmaps.readthedocs.io/en/latest/api.html

markers = gmaps.marker_layer(hotel_coord, info_box_content = hotel_info)

# Add the marker layer to the map where they can be clicked to show info box data.

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))